In [66]:
import numpy as np
import math
import torch
import matplotlib.pyplot as plt
import gymnasium as gym
from gym import Env
from gym import spaces
import random
from IPython.display import clear_output
import os
import matplotlib
from collections import namedtuple, deque
from itertools import count
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from pettingzoo import ParallelEnv
import seaborn as sns

is_ipython = 'inline' in matplotlib.get_backend()
if is_ipython:
    from IPython import display

plt.ion()

#device setup
device = "cpu"

import environment
from seq_buffer import Transition, SequentialReplayBuffer

In [67]:
freq_agile_radar_versions = [[9 * math.pow(10, 9), 9.4* math.pow(10, 9)], [8 * math.pow(10, 9), 8.5* math.pow(10, 9)], [6 * math.pow(10, 9), 6.2* math.pow(10, 9)]]

hop_states = []

for i in range (0,3):
    radar_states = []
    interval = (freq_agile_radar_versions[i][1] - freq_agile_radar_versions[i][0])/10
    for j in range(0,24):
        radar_states.append(int(freq_agile_radar_versions[i][0]+ interval * j))
    hop_states.append(radar_states)

jamming_bandwidths = [10e6, 25e6, 40e6]

In [68]:
env = environment.ParallelRadarJammingEnv(hop_states, jamming_bandwidths)

In [69]:
class FocalLoss(nn.Module):
    def __init__(self, alpha=0.25, gamma=2.0, reduction='mean'):
        super(FocalLoss, self).__init__()
        self.alpha = alpha
        self.gamma = gamma
        self.reduction = reduction

    def forward(self, inputs, targets):
        # Flatten inputs and targets for easier computation
        inputs = inputs.view(-1)
        targets = targets.view(-1)
        
        # Cross-entropy loss
        ce_loss = nn.BCELoss(reduction='none')(inputs, targets)
        
        # Compute the focal loss components
        pt = torch.exp(-ce_loss)  # pt is the probability of the true class
        focal_loss = self.alpha * (1 - pt) ** self.gamma * ce_loss
        
        if self.reduction == 'mean':
            return focal_loss.mean()
        elif self.reduction == 'sum':
            return focal_loss.sum()
        else:
            return focal_loss

In [70]:
class Autoencoder(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(Autoencoder, self).__init__()
        # Encoder
        self.encoder = nn.Sequential(
            nn.Linear(input_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, hidden_size),
            nn.ReLU()
        )
        # Decoder
        self.decoder = nn.Sequential(
            nn.Linear(hidden_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, input_size),
            nn.Sigmoid()  # Sigmoid to match binary input/output
        )

    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded

In [71]:
class AgentWithAutoencoder:
    def __init__(self, agent_name, autoencoder, optimizer, focal_loss, device):
        self.agent_name = agent_name
        self.autoencoder = autoencoder  # Autoencoder model for the agent
        self.optimizer = optimizer  # Optimizer (e.g., Adam)
        self.focal_loss = focal_loss  # Focal Loss function
        self.device = device  # Device (CPU/GPU)
    
    def train_on_observation(self, observation):
        """
        Train the autoencoder model on a single observation.
        The focal loss is used to compute the loss between the reconstructed and actual observations.
        """
        # Convert the observation to a tensor and move to the appropriate device
        observation_tensor = torch.tensor(observation, dtype=torch.float32).to(self.device).unsqueeze(0)  # Add batch dimension

        # Zero the gradients from the previous step
        self.optimizer.zero_grad()

        # Forward pass through the autoencoder to get the reconstructed observation
        reconstructed_observation = self.autoencoder(observation_tensor)

        # Compute the Focal Loss between the reconstructed and the original observation
        loss = self.focal_loss(reconstructed_observation, observation_tensor)

        # Backpropagate the loss and optimize the autoencoder's parameters
        loss.backward()
        self.optimizer.step()

        return loss.item()  # Return the loss value for tracking

In [77]:
def train_multi_agent(agents, env, num_episodes=1000, epsilon_start=1.0, epsilon_end=0.1, epsilon_decay_rate=0.998, warmup_episodes=100):
    """
    Train multiple agents in parallel using an epsilon-greedy policy with a warmup process.
    
    Parameters:
    - agents: Dictionary of agents (AgentWithAutoencoder instances).
    - env: The multi-agent reinforcement learning environment.
    - num_episodes: Total number of training episodes.
    - epsilon_start: Initial epsilon value for the epsilon-greedy policy.
    - epsilon_end: Minimum epsilon value for decay.
    - epsilon_decay_rate: Rate of exponential decay for epsilon.
    - warmup_episodes: Number of episodes used for warmup (no training).
    """
    epsilon = epsilon_start

    for episode in range(num_episodes):
        total_rewards = {agent_name: 0 for agent_name in env.agents}
        observations = env.reset()
        done = {agent_name: False for agent_name in env.agents}

        while not all(done.values()):
            actions = {}

            # Select actions for each agent
            for agent_name, observation in observations.items():

                if np.random.rand() < epsilon:
                    action = env.action_space[agent_name].sample()
                else:
                    obs_tensor = torch.tensor(observation, dtype=torch.float32, device=agents[agent_name].device).unsqueeze(0)
                    reconstructed_obs = agents[agent_name].autoencoder(obs_tensor)
                    action = torch.argmax(reconstructed_obs).item()

                actions[agent_name] = action

            # Step the environment
            next_observations, rewards, done, _ = env.step(actions)

            # Store experiences and optionally train
            for agent_name, (observation, action, reward, next_observation) in zip(
                env.agents, zip(observations.values(), actions.values(), rewards.values(), next_observations.values())
            ):

                agents[agent_name].train_on_observation(observation)
                total_rewards[agent_name] += reward

            observations = next_observations

        # Decay epsilon
        epsilon = max(epsilon_end, epsilon * epsilon_decay_rate)

        # Log performance
        total_reward_sum = sum(total_rewards.values())
        print(f"Episode {episode} Total Reward: {total_reward_sum}")

# Initialize six agents
agents = {}
for agent_name in env.agents:
    autoencoder = Autoencoder(input_size = sum(env.n_frequencies), hidden_size=32).to(device)
    optimizer = optim.Adam(autoencoder.parameters(), lr=0.0001)
    focal_loss = FocalLoss(alpha=0.9, gamma=1.0)
    agents[agent_name] = AgentWithAutoencoder(
        agent_name=agent_name,
        autoencoder=autoencoder,
        optimizer=optimizer,
        focal_loss=focal_loss,
        device=device,
    )

# Train the agents
train_multi_agent(agents, env, num_episodes=1000)

Episode 0 Total Reward: -43260
Episode 1 Total Reward: -43460
Episode 2 Total Reward: -47490
Episode 3 Total Reward: -43150
Episode 4 Total Reward: -33820
Episode 5 Total Reward: -32190
Episode 6 Total Reward: -49370
Episode 7 Total Reward: -49730
Episode 8 Total Reward: -58210
Episode 9 Total Reward: -53750
Episode 10 Total Reward: -44310
Episode 11 Total Reward: -38240
Episode 12 Total Reward: -45840
Episode 13 Total Reward: -54710
Episode 14 Total Reward: -48470
Episode 15 Total Reward: -49300
Episode 16 Total Reward: -40140
Episode 17 Total Reward: -51230
Episode 18 Total Reward: -51390
Episode 19 Total Reward: -41430
Episode 20 Total Reward: -45630
Episode 21 Total Reward: -49360
Episode 22 Total Reward: -54140
Episode 23 Total Reward: -44430
Episode 24 Total Reward: -41190
Episode 25 Total Reward: -42480
Episode 26 Total Reward: -30230
Episode 27 Total Reward: -43870
Episode 28 Total Reward: -44980
Episode 29 Total Reward: -58330
Episode 30 Total Reward: -38230
Episode 31 Total R

KeyboardInterrupt: 

In [ ]:
def train_multi_agent_no_epsilon(agents, env, num_episodes=1000, warmup_episodes=10):
    """
    Train multiple agents in parallel without using an epsilon-greedy policy.
    
    Parameters:
    - agents: Dictionary of agents (AgentWithAutoencoder instances).
    - env: The multi-agent reinforcement learning environment.
    - num_episodes: Total number of training episodes.
    - warmup_episodes: Number of episodes used for warmup (no training).
    """
    for episode in range(num_episodes):
        total_rewards = {agent_name: 0 for agent_name in env.agents}
        observations = env.reset()
        done = {agent_name: False for agent_name in env.agents}

        while not all(done.values()):
            actions = {}

            # Select actions for each agent
            for agent_name, observation in observations.items():
                if episode < warmup_episodes:
                    # During warmup, use random actions
                    action = env.action_space[agent_name].sample()
                else:
                    # Select action based on the agent's autoencoder
                    obs_tensor = torch.tensor(observation, dtype=torch.float32, device=agents[agent_name].device).unsqueeze(0)
                    reconstructed_obs = agents[agent_name].autoencoder(obs_tensor)
                    action = torch.argmax(reconstructed_obs).item()

                actions[agent_name] = action

            # Step the environment
            next_observations, rewards, done, _ = env.step(actions)

            # Store experiences and optionally train
            for agent_name, (observation, action, reward, next_observation) in zip(
                env.agents, zip(observations.values(), actions.values(), rewards.values(), next_observations.values())
            ):
                if episode >= warmup_episodes:
                    # Train the agent only after the warmup phase
                    agents[agent_name].train_on_observation(observation)
                total_rewards[agent_name] += reward

            observations = next_observations

        # Log performance
        total_reward_sum = sum(total_rewards.values())
        print(f"Episode {episode} Total Reward: {total_reward_sum:.4f}")

        # Debug: Monitor autoencoder loss periodically
        if episode % 50 == 0 and episode >= warmup_episodes:
            for agent_name, agent in agents.items():
                print(f"Agent {agent_name} Training Update Complete")

# Train the agents with the updated loop
train_multi_agent_no_epsilon(agents, env, num_episodes=1000, warmup_episodes=10)